In [1]:
from model.types import MarketDemandSupply
from model.run import load_config, run, post_processing

from model.mechanism.amm_k import amm_k_mechanism
from model.mechanism.treasury import liq_backing_mechanism,treasury_stables_mechanism,floating_supply_mechanism,liq_ohm_mechanism
from model.policy.rbs_price import lower_target_policy,upper_target_policy
initialstate = { # variable values borrowed from liquidity-olympus/simulation.ipynb
            'supply':30000000,
            "liq_stables": 21000000,
            "reserves_stables":200000000,
            "reserves_volatile":17000000,
            "price":9,

            "market_demand_supply": MarketDemandSupply(total_supply = 50,
                                                   total_demand = 50),
                                                   
            # treasury policy
            'target_liq_ratio_reached':False,
            'reserves_in':0,
            'supply':25000000,

            # RBS related
            'ma_target':9,
            # market transaction variables
            'ask_change_ohm':0,
            'bid_change_ohm':0,
            "net_flow": None
            }
# initialize parameters that are decided by other parameters
initialstate['amm_k'] = amm_k_mechanism(initialstate)
initialstate['price_history']=[initialstate['price']]

initialstate['lb_target'] = initialstate['ma_target']
initialstate['price_target']=initialstate['ma_target']

initialstate['liq_ohm'] = liq_ohm_mechanism(initialstate['liq_stables'],initialstate['price'])
initialstate['floating_supply'] = floating_supply_mechanism(initialstate['supply'],initialstate['liq_ohm'])

params = {"demand_factor": [10],
         "supply_factor": [10],
         "initial_reserves_volatile":[17000000], # setting it as constant since in this simulation we don't use it at all.
         "reward_rate":[0.000198], # flat reward rate; (1 + r) ^ 365 ~ 7.5%
         "max_liq_ratio":[0.14375],  # LiquidityUSD : reservesUSD ratio --> 1:1 = 0.5
         "target_ma":[30], # number of days
         "lower_wall":[0.15],
         "upper_wall":[0.15],
         "lower_cushion":[.075],
         "upper_cushion":[.075],
         }

initialstate['treasury_stables'] = treasury_stables_mechanism(initialstate['liq_stables'],initialstate['reserves_stables'])
initialstate['liq_backing'] = liq_backing_mechanism(initialstate['treasury_stables'],params['initial_reserves_volatile'][0])
initialstate['target_walls'] = [lower_target_policy(initialstate['price_target'], params['lower_wall'][0]),
                                upper_target_policy(initialstate['price_target'], params['upper_wall'][0])]
initialstate['target_cushions'] = [lower_target_policy(initialstate['price_target'], params['lower_cushion'][0]),
                                    upper_target_policy(initialstate['price_target'], params['upper_cushion'][0])]
initialstate['bid_counter'],initialstate['ask_counter'] = [],[]

nrun = 5
nstep = 10
exp = load_config(nrun, params, initialstate, nstep)
raw = run(exp)
df = post_processing(raw)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 10, 5, 24)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 10, 5, 24)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 0.42s


In [2]:
raw.columns

Index(['supply', 'liq_stables', 'reserves_stables', 'reserves_volatile',
       'price', 'market_demand_supply', 'target_liq_ratio_reached',
       'reserves_in', 'ma_target', 'ask_change_ohm', 'bid_change_ohm',
       'net_flow', 'amm_k', 'price_history', 'lb_target', 'price_target',
       'liq_ohm', 'floating_supply', 'treasury_stables', 'liq_backing',
       'target_walls', 'target_cushions', 'bid_counter', 'ask_counter',
       'simulation', 'subset', 'run', 'substep', 'timestep'],
      dtype='object')

# Check Summary

1. Changed s_liq_backing to use the state instead of parameters for reserves_volatile.
2. Ordering needs to be changed of substeps

In [3]:
left = raw[raw["substep"] == 1][['liq_stables', 'reserves_stables']].sum(axis=1)
right = raw[raw["substep"] == 1]['treasury_stables']

assert (left == right).all()

In [4]:
left = raw[raw["substep"] == 2][['liq_stables', 'reserves_stables', 'reserves_volatile']].sum(axis=1)
right = raw[raw["substep"] == 2]['liq_backing']

assert (left == right).all()

In [5]:
left = raw[raw["substep"] == 3]['price_history'].apply(lambda x: len(x))
right = raw[raw["substep"] == 3]['timestep'] + 1

assert (left == right).all()

In [6]:
raw['bid_counter']

0       []
1       []
2       []
3       []
4      [0]
      ... 
600    [0]
601    [0]
602    [0]
603    [0]
604    [0]
Name: bid_counter, Length: 605, dtype: object

In [7]:
raw['ask_counter']

0       []
1       []
2       []
3       []
4       []
      ... 
600    [1]
601    [1]
602    [1]
603    [1]
604    [1]
Name: ask_counter, Length: 605, dtype: object